In [1]:
# !pip install sranodec

In [1]:
import pandas as pd
import numpy as np
import main_classificaiton as mc
import torch
import torch.nn as nn
import torch.optim as optim
import pickle
import os
import random


In [2]:
# seed 고정
random_seed = 42

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [3]:
# Case 1. w/o data representation & RNN model 
config1 = {
        'with_representation': False, # classification에 사용되는 representation이 있을 경우 True, 아닐 경우 False
        'algorithm': 'RNN', # classification에에 활용할 알고리즘 정의, {'RNN', 'LSTM', 'GRU', 'CNN_1D', 'FC_layer'} 중 택 1

        'alg_parameter': {
            'window_size' : 50, # input time series data를 windowing 하여 자르는 길이(size)
            'num_layers' : 2, # recurrnet layers의 수, Default : 1
            'hidden_size' : 64, # hidden state의 벡터차원 수
            'attention' : False, # True일 경우 attention layer를 추가
            'dropout' : 0.2, # If non-zero, introduces a Dropout layer on the outputs of each RNN layer except the last layer, with dropout probability equal to dropout. Default: 0
            'bidirectional' : True, # 모델의 양방향성 여부
            'batch_size' : 64, #batch size
            'device': 'cuda', # 학습 환경, ["cuda", "cpu"] 중 선택
            'num_epochs' : 100 # 학습 시 사용할 epoch 수
            }
}

In [4]:
# Case 2. w/o data representation &LSTM model 
config2 = {
        'with_representation': False, # classification에 사용되는 representation이 있을 경우 True, 아닐 경우 False
        'algorithm': 'LSTM', # classification에에 활용할 알고리즘 정의, {'RNN', 'LSTM', 'GRU', 'CNN_1D', 'FC_layer'} 중 택 1

        'alg_parameter': {
            'window_size' : 50, # input time series data를 windowing 하여 자르는 길이(size)
            'num_layers' : 2, # recurrnet layers의 수, Default : 1
            'hidden_size' : 64, # hidden state의 벡터차원 수
            'attention' : False, # True일 경우 attention layer를 추가
            'dropout' : 0.2, # If non-zero, introduces a Dropout layer on the outputs of each RNN layer except the last layer, with dropout probability equal to dropout. Default: 0
            'bidirectional' : True, # 모델의 양방향성 여부
            'batch_size' : 64, #batch size
            'device': 'cuda', # 학습 환경, ["cuda", "cpu"] 중 선택
            'num_epochs' : 100 # 학습 시 사용할 epoch 수
            }
}

In [5]:
# Case 3. w/o data representation & GRU model 
config3 = {
        'with_representation': False, # classification에 사용되는 representation이 있을 경우 True, 아닐 경우 False
        'algorithm': 'GRU', # classification에에 활용할 알고리즘 정의, {'RNN', 'LSTM', 'GRU', 'CNN_1D', 'FC_layer'} 중 택 1

        'alg_parameter': {
            'window_size' : 50, # input time series data를 windowing 하여 자르는 길이(size)
            'num_layers' : 2, # recurrnet layers의 수, Default : 1
            'hidden_size' : 64, # hidden state의 벡터차원 수
            'attention' : False, # True일 경우 attention layer를 추가
            'dropout' : 0.2, # If non-zero, introduces a Dropout layer on the outputs of each RNN layer except the last layer, with dropout probability equal to dropout. Default: 0
            'bidirectional' : True, # 모델의 양방향성 여부
            'batch_size' : 64, #batch size
            'device': 'cuda', # 학습 환경, ["cuda", "cpu"] 중 선택
            'num_epochs' : 100 # 학습 시 사용할 epoch 수
            }
}

In [6]:
# Case 4. w/o data representation & CNN_1D model 
config4 = {
        'with_representation': False, # classification에 사용되는 representation이 있을 경우 True, 아닐 경우 False
        'algorithm': 'CNN_1D', # classification에에 활용할 알고리즘 정의, {'RNN', 'LSTM', 'GRU', 'CNN_1D', 'FC_layer'} 중 택 1

        'alg_parameter': {
            'window_size' : 50, # input time series data를 windowing 하여 자르는 길이(size)
            'output_channels' : 64, # convolution channel size of output
            'drop_out' : 0.2, # If non-zero, introduces a Dropout layer on the outputs of each RNN layer except the last layer, with dropout probability equal to dropout. Default: 0
            'kernel_size' : 3, # convolutional filter size
            'stride' : 1, # stride of the convolution. Default = 1 
            'padding' : 0, # padding added to both sides of the input. Default = 0
            'batch_size' : 64, # batch size
            'device': 'cuda', # 학습 환경, ["cuda", "cpu"] 중 선택
            'num_epochs' : 100 # 학습 시 사용할 epoch 수
            }
}

In [7]:
# Case 5. w data representation & fully-connected layers 
# 특징 벡터의 사이즈 = 20 이라고 가정

config5 = {
        'with_representation': True, # classification에 사용되는 representation이 있을 경우 True, 아닐 경우 False
        'algorithm': 'FC_Layer', # classification에에 활용할 알고리즘 정의, {'RNN', 'LSTM', 'GRU', 'CNN_1D', 'FC_Layer'} 중 택 1

        'alg_parameter': {
            'window_size' : 50, # input time series data를 windowing 하여 자르는 길이(size)
            'input_representation' : 0, # 예를 들면 (80, 20) 차원의 벡터 (80은 window_length에 따른 관측치 수, 20은 representation 특징벡터 차원 수)를 넣어야 함. 지금은 loader부분에서 random값들어가 있음
            'drop_out' : 0.2, # If non-zero, introduces a Dropout layer on the outputs of each RNN layer except the last layer, with dropout probability equal to dropout. Default: 0
            'batch_size' : 64, # batch size
            'bias': True, # bias [True, False]
            'device': 'cuda', # 학습 환경, ["cuda", "cpu"] 중 선택
            'num_epochs' : 100 # 학습 시 사용할 epoch 수
            }
}

In [8]:
data_dir = './data'

train_x = pickle.load(open(os.path.join(data_dir, 'x_train.pkl'), 'rb'))
train_y = pickle.load(open(os.path.join(data_dir, 'state_train.pkl'), 'rb'))
test_x =  pickle.load(open(os.path.join(data_dir, 'x_test.pkl'), 'rb'))
test_y = pickle.load(open(os.path.join(data_dir, 'state_test.pkl'), 'rb'))

# train_data = {'x' : train_x, 'y' : train_y}
# test_data = {'x' : test_x, 'y' : test_y}

In [9]:
# Case 1. w/o data representation & RNN
config = config1
data_classification = mc.Classification(config, train_x, train_y, test_x, test_y)
test_accuracy = data_classification.getResult()

print('\naccuracy of test set : {}'.format(test_accuracy))

Epoch 1/100
----------
train Loss: 1.7534 Acc: 0.1250
val Loss: 1.6363 Acc: 0.2400

Epoch 2/100
----------
train Loss: 1.6556 Acc: 0.2125
val Loss: 1.5519 Acc: 0.4400

Epoch 3/100
----------
train Loss: 1.5757 Acc: 0.4125
val Loss: 1.4809 Acc: 0.5200

Epoch 4/100
----------
train Loss: 1.5100 Acc: 0.4250
val Loss: 1.4213 Acc: 0.5200

Epoch 5/100
----------
train Loss: 1.4546 Acc: 0.4375
val Loss: 1.3730 Acc: 0.5200

Epoch 6/100
----------
train Loss: 1.4105 Acc: 0.4125
val Loss: 1.3356 Acc: 0.5200

Epoch 7/100
----------
train Loss: 1.3754 Acc: 0.4250
val Loss: 1.3068 Acc: 0.5200

Epoch 8/100
----------
train Loss: 1.3448 Acc: 0.4250
val Loss: 1.2837 Acc: 0.5200

Epoch 9/100
----------
train Loss: 1.3204 Acc: 0.4250
val Loss: 1.2664 Acc: 0.5200

Epoch 10/100
----------
train Loss: 1.2967 Acc: 0.4500
val Loss: 1.2545 Acc: 0.5200

Epoch 11/100
----------
train Loss: 1.2747 Acc: 0.4625
val Loss: 1.2456 Acc: 0.5200

Epoch 12/100
----------
train Loss: 1.2561 Acc: 0.5000
val Loss: 1.2390 Ac

In [10]:
# Case 2. w/o data representation & LSTM
config = config2
data_classification = mc.Classification(config, train_x, train_y, test_x, test_y)
test_accuracy = data_classification.getResult()

print('\naccuracy of test set : {}'.format(test_accuracy))

Epoch 1/100
----------
train Loss: 1.8122 Acc: 0.0125
val Loss: 1.7687 Acc: 0.4800

Epoch 2/100
----------
train Loss: 1.7889 Acc: 0.3500
val Loss: 1.7440 Acc: 0.6800

Epoch 3/100
----------
train Loss: 1.7664 Acc: 0.5125
val Loss: 1.7192 Acc: 0.6400

Epoch 4/100
----------
train Loss: 1.7452 Acc: 0.5125
val Loss: 1.6954 Acc: 0.6000

Epoch 5/100
----------
train Loss: 1.7235 Acc: 0.5125
val Loss: 1.6731 Acc: 0.6000

Epoch 6/100
----------
train Loss: 1.7014 Acc: 0.5125
val Loss: 1.6515 Acc: 0.6000

Epoch 7/100
----------
train Loss: 1.6814 Acc: 0.5000
val Loss: 1.6300 Acc: 0.6000

Epoch 8/100
----------
train Loss: 1.6604 Acc: 0.5000
val Loss: 1.6089 Acc: 0.6000

Epoch 9/100
----------
train Loss: 1.6393 Acc: 0.5000
val Loss: 1.5880 Acc: 0.6000

Epoch 10/100
----------
train Loss: 1.6196 Acc: 0.5000
val Loss: 1.5670 Acc: 0.6000

Epoch 11/100
----------
train Loss: 1.5990 Acc: 0.5000
val Loss: 1.5464 Acc: 0.6000

Epoch 12/100
----------
train Loss: 1.5786 Acc: 0.5125
val Loss: 1.5257 Ac

In [11]:
# Case 3. w/o data representation & GRU
config = config3
data_classification = mc.Classification(config, train_x, train_y, test_x, test_y)
test_accuracy = data_classification.getResult()

print('\naccuracy of test set : {}'.format(test_accuracy))

Epoch 1/100
----------
train Loss: 1.8700 Acc: 0.0000
val Loss: 1.8108 Acc: 0.0400

Epoch 2/100
----------
train Loss: 1.8000 Acc: 0.1000
val Loss: 1.7409 Acc: 0.2400

Epoch 3/100
----------
train Loss: 1.7355 Acc: 0.2875
val Loss: 1.6773 Acc: 0.4800

Epoch 4/100
----------
train Loss: 1.6768 Acc: 0.4000
val Loss: 1.6191 Acc: 0.5200

Epoch 5/100
----------
train Loss: 1.6251 Acc: 0.4125
val Loss: 1.5679 Acc: 0.5200

Epoch 6/100
----------
train Loss: 1.5756 Acc: 0.4250
val Loss: 1.5228 Acc: 0.5200

Epoch 7/100
----------
train Loss: 1.5337 Acc: 0.4250
val Loss: 1.4808 Acc: 0.5200

Epoch 8/100
----------
train Loss: 1.4958 Acc: 0.4250
val Loss: 1.4427 Acc: 0.5200

Epoch 9/100
----------
train Loss: 1.4605 Acc: 0.4250
val Loss: 1.4082 Acc: 0.5200

Epoch 10/100
----------
train Loss: 1.4308 Acc: 0.4250
val Loss: 1.3761 Acc: 0.5200

Epoch 11/100
----------
train Loss: 1.4047 Acc: 0.4250
val Loss: 1.3470 Acc: 0.5200

Epoch 12/100
----------
train Loss: 1.3812 Acc: 0.4250
val Loss: 1.3215 Ac

In [12]:
# Case 4. w/o data representation & CNN_1D
config = config4
data_classification = mc.Classification(config, train_x, train_y, test_x, test_y)
test_accuracy = data_classification.getResult()

print('\naccuracy of test set : {}'.format(test_accuracy))

Epoch 1/100
----------
train Loss: 1.7814 Acc: 0.2500
val Loss: 1.7544 Acc: 0.1200

Epoch 2/100
----------
train Loss: 1.7095 Acc: 0.2875
val Loss: 1.6926 Acc: 0.1200

Epoch 3/100
----------
train Loss: 1.6575 Acc: 0.3125
val Loss: 1.6356 Acc: 0.1200

Epoch 4/100
----------
train Loss: 1.5843 Acc: 0.3875
val Loss: 1.5825 Acc: 0.2400

Epoch 5/100
----------
train Loss: 1.5424 Acc: 0.4125
val Loss: 1.5268 Acc: 0.5600

Epoch 6/100
----------
train Loss: 1.4902 Acc: 0.3500
val Loss: 1.4675 Acc: 0.5600

Epoch 7/100
----------
train Loss: 1.4291 Acc: 0.4625
val Loss: 1.4075 Acc: 0.5600

Epoch 8/100
----------
train Loss: 1.3764 Acc: 0.4375
val Loss: 1.3494 Acc: 0.5200

Epoch 9/100
----------
train Loss: 1.3315 Acc: 0.4250
val Loss: 1.3014 Acc: 0.5200

Epoch 10/100
----------
train Loss: 1.2848 Acc: 0.4250
val Loss: 1.2664 Acc: 0.5200

Epoch 11/100
----------
train Loss: 1.2473 Acc: 0.4125
val Loss: 1.2448 Acc: 0.5200

Epoch 12/100
----------
train Loss: 1.1952 Acc: 0.4375
val Loss: 1.2327 Ac

In [15]:
# Case 5. w/ data representation & fully-connected layers
config = config5
data_classification = mc.Classification(config, train_x, train_y, test_x, test_y)
test_accuracy = data_classification.getResult()

print('\naccuracy of test set : {}'.format(test_accuracy))

Epoch 1/100
----------
train Loss: 1.7527 Acc: 0.2500
val Loss: 1.7426 Acc: 0.3200

Epoch 2/100
----------
train Loss: 1.7594 Acc: 0.2375
val Loss: 1.7420 Acc: 0.3200

Epoch 3/100
----------
train Loss: 1.7400 Acc: 0.3000
val Loss: 1.7414 Acc: 0.3200

Epoch 4/100
----------
train Loss: 1.7397 Acc: 0.2750
val Loss: 1.7408 Acc: 0.3200

Epoch 5/100
----------
train Loss: 1.7663 Acc: 0.2500
val Loss: 1.7402 Acc: 0.3200

Epoch 6/100
----------
train Loss: 1.7421 Acc: 0.2625
val Loss: 1.7396 Acc: 0.3200

Epoch 7/100
----------
train Loss: 1.7351 Acc: 0.2625
val Loss: 1.7390 Acc: 0.3200

Epoch 8/100
----------
train Loss: 1.7468 Acc: 0.2625
val Loss: 1.7384 Acc: 0.3200

Epoch 9/100
----------
train Loss: 1.7393 Acc: 0.2500
val Loss: 1.7378 Acc: 0.3200

Epoch 10/100
----------
train Loss: 1.7500 Acc: 0.2375
val Loss: 1.7372 Acc: 0.3200

Epoch 11/100
----------
train Loss: 1.7356 Acc: 0.3375
val Loss: 1.7367 Acc: 0.3200

Epoch 12/100
----------
train Loss: 1.7323 Acc: 0.3000
val Loss: 1.7362 Ac